In [1]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
import pandas_gbq as pbq
import numpy as np
from noaa.ghcnd import NOAAStore

In [2]:
store = NOAAStore()

In [3]:
stations = store.stations()

In [4]:
tmin = store.timeseries_universe(element='TMIN')

In [4]:
prcp = store.timeseries_universe(element='PRCP')

In [ ]:
inv = store.stations_inventory(element='PRCP', include_info=True)
inv_active = inv.loc[lambda df:(df['lastyear']>=2022)].copy()
inv_active_1950 = inv_active.loc[lambda df:(df['firstyear']<1950)].groupby('country')['country'].count().sort_values(ascending=False)
inv_active_1950.head(10)

In [ ]:
prcp = store.timeseries_universe(element='PRCP')

In [ ]:
count_1950 = prcp.loc['1950-01-01':].groupby('id').count()
count_1950_90pct = count_1950[(count_1950/count_1950.max()>=0.9)]

In [ ]:
prcp_filtered = prcp.loc[slice(None), count_1950_90pct.index.values].copy()
# prcp_filtered_2 = prcp_filtered.copy().reorder_levels(['id','date']).sort_index()

In [ ]:
def coverage(s):
    start = s.index.get_level_values('date').min()
    end = s.index.get_level_values('date').max()
    count = s.count()
    length = (end-start).days+1
    coverage = count/length
    return {'start':start,'end':end,'coverage':coverage}
    

In [ ]:
prcp_stats = pd.DataFrame(prcp.groupby('id').apply(coverage)).unstack(1)['value'].sort_values('coverage')
prcp_stats

In [ ]:
prcp_stats.loc[lambda df:(df['start']<='1950-01-01')&(df['end']>='2022-01-01')&(df['coverage']>=0.90)].sort_values('start')

In [2]:
import pandas_gbq as pbq
from google.cloud import bigquery

In [3]:
element = 'PRCP'

qry = f"""
SELECT data_daily.date, data_daily.id, data_daily.value
FROM `bigquery-public-data.ghcn_d.ghcnd_*` AS data_daily
JOIN `bigquery-public-data.ghcn_d.ghcnd_inventory` AS inventory
    ON data_daily.id = inventory.id
    AND inventory.element = '{element}'
    AND inventory.firstyear <= 1975
    AND inventory.lastyear >= 2022
    AND inventory.latitude BETWEEN 39.0 AND 42.0
    AND inventory.longitude BETWEEN -76.0 AND -72.0
WHERE REGEXP_CONTAINS(_TABLE_SUFFIX, '^(19|20)[0-9]{{2}}$')
    AND data_daily.element = '{element}'
    AND data_daily.date >= '1970-01-01'
    AND data_daily.id LIKE 'US%'
ORDER BY data_daily.date, data_daily.id
"""

print(qry)
# df = pbq.read_gbq(qry)


SELECT data_daily.date, data_daily.id, data_daily.value
FROM `bigquery-public-data.ghcn_d.ghcnd_*` AS data_daily
JOIN `bigquery-public-data.ghcn_d.ghcnd_inventory` AS inventory
    ON data_daily.id = inventory.id
    AND inventory.element = 'PRCP'
    AND inventory.firstyear <= 1975
    AND inventory.lastyear >= 2022
    AND inventory.latitude BETWEEN 39.0 AND 42.0
    AND inventory.longitude BETWEEN -76.0 AND -72.0
WHERE REGEXP_CONTAINS(_TABLE_SUFFIX, '^(19|20)[0-9]{2}$')
    AND data_daily.element = 'PRCP'
    AND data_daily.date >= '1970-01-01'
    AND data_daily.id LIKE 'US%'
ORDER BY data_daily.date, data_daily.id

